#### Measuring the Impact of News Headlines on Stock Movements

In [ ]:
import os
import re
import glob
from pathlib import Path
import pandas as pd
import numpy as np
import yfinance as yf
from useful.eda import basic_info

FAT_BAR = '='*50

##### Abstract

The Dow Jones Industrial Average (DJIA) measures the performance of 30 companies (https://money.cnn.com/data/dow30/) and some consider it to be an indicator of the American economy. In general the market price of the index is a reflection of investors attitudes about it's future direction. If the market is rising and expected to rise then we call this a bull market, and in the opposite situation we call it a bear market.  

There have been many attempts to measure investor sentiment as it could be indicative of the market's direction and thus assist in trading strategies which consist of fundamentally buying or selling. Although the market behaves randomly there may be ways to predict market movements in special situations. 

Given that the market reacts to investors attitudes we can speculate that the news either  has an effect on market's price or that it moves the market itself. We cannot be certain which of the two if any, causes market movements and we must be very careful not to say that whenever headline A happens market movement B will happen. However, my suspicision is that under certain market circumstances we can make better guesses at what will happen.

The goal of this notebook is to train a supervised model to predict whether the market will go up or down over a certain period. To accomplish this we will take a set of headlines from 2008 to 2015 and match it with the closing price of the Down Jones Industrial Average from that same time period. We will use a natural language technique called bag of words, expressed through sci-kit learn's count-vectorize library to create the feature set for this model. 

##### The Data

The stock data comes from yfiance (https://pypi.org/project/yfinance/), and we are using the DJIA index. The DJIA index consists of Open, High, Low, Close, and Volume. For our purposes we will use Close as it is considered the standard benchmark for performance overtime (https://www.investopedia.com/terms/c/closingprice.asp). Stock data are discrete random variables and the index goes out to two decimal places. The data source I have obtained only goes out to one decimal place. It should be noted that the actual return on the stock is considered a continuous random variable because it can have an infinite number of possible returns in theory.

The news data set comes from reddit news headlines. It is worthy noting that the news data has multiple different headlines for the same day, as opposed to the stock data which has only one closing price. The news headlines contain strings of the headlines and will be transfomed into vectors in which each row will be a news headlines and each column will be a feature (word or bi-gram). 

The data will be converted to feather format from csv. For more info on the feather format please visit: https://github.com/wesm/feather. 

##### Import the data and convert to feather

In [ ]:
#get data file directory
DATA_DIR = os.path.join(str(Path.cwd().parent)+'/data/')

In [ ]:
#unzip only csv file for reddit headlines
#TODO - if file is present unzip else leave
for f in [f for f in os.listdir(DATA_DIR) if f.endswith('.zip')]:
    ! cd .. && unzip data/{f} -d data/ && cd notebooks

In [ ]:
#convert csv --> feather
for f in [f for f in os.listdir(DATA_DIR) if f.endswith('.csv')]:
    print(f'loading {f}')
    df = pd.read_csv(os.path.join(DATA_DIR, f))
    df.to_feather(os.path.join(DATA_DIR, f"{f.replace('.csv', '')}.feather"))

In [ ]:
files = list(filter(lambda x: x.endswith('.feather'), os.listdir(DATA_DIR)))

In [ ]:
#pull all the dfs into a dictionary for future use & print out basic data
data = {i.split('.')[0]: pd.read_feather(
            os.path.join(
                DATA_DIR + '/' + [x for x in files if x.startswith(i.split('.')[0])][0]
            )
        ) for i in files}

news_df = data['RedditNews']
news_df = news_df.set_index('Date') 

news_df = news_df.iloc[::-1]

basic_info.data_info(news_df,None)

###### News headlines

As you can see above the news headlines data consists of the date and a column for news columns. The news column is an object (string) and the date is the index. If the date is in object format we can handle this later through pandas datetime. There is no missing data, and it looks like there are about 71 redudant headlines, with only one headline occuring 3 times. 

##### DJIA data

The DJIA data is downloaded from the yfinance library and I set the dates from 2008-06-08 to 2016-07-01 to match with the news headlines. If there is some overlap this will be corrected when I merge the datasets.

There are 1820 floats in 5 columns consisting of Open, High, Low, Close, Adj Close, and Volume. There is no missing data and no duplicates. 

The dates covered in this dataset are significant in that they cover the 2008 recission and post 2008 recovery. 

In [ ]:
dji = yf.download('DJI')

In [ ]:
dji_df = dji[(dji.index >= '2008-06-08') & (dji.index <= '2016-07-01')]

basic_info.data_info(dji_df,None)

##### Graph of the DJIA closing price from 2008-06-08 to 2016-07-01

This is a plot of the closing price from June 2008 to July 2016...

### Convert to date

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(20, 6))

sns.tsplot(data=dji_df.loc[:,'Close'])
plt.title('Dow Jones closing price from 2008-06-08 - 2016-07-01')
plt.ylabel('Closing Price')
plt.xlabel('Days')
plt.show()

#### Dow Jones Feature engineering: Prediction Horizon

At the heart of this project is to track the predictive ability of news headlines. To do so we need to see if a news headline effects the price of the DJIA at some point in the future. To begin this exericise I chose a 10 day horizon. All we are tracking for our variable is whether the stock went up or down on a n-day horizon. For example if we were to take a price on day 0 and measure it against day 10, if that price on day 10 were high than day one we would assign that day 0 date a 1 (up). If day 10 were lower than day 0, we assign the day 0 date a 0 (down). It is not clear what is the best prediction horizon we should use so I arbitrarily chose 10 days. I will use 5 days and 15 as well to see if we can obtain better results. To limit the scope of this notebook I will limit the prediction horizon to these days. 

##### Engineering the prediction horizon - further details...

The goal is to match up a headline with a closing price n days in the future. The assumption being that the effects of the events underlying the headline will manifest itself n days in the future. For example if we use a 10 day prediction horizon the closing price in our shifed dataframe for 2008-06-09 will be from 2008-06-25 (10 days in the future based on the dates we have in our dataset). Therefore when we merge the stock price with the headline dataframe the algorithm will be trained on that day's headline and a price n days in the future. Our final step will be to assign a value (0 or 1) corresponding to whether the market goes up or down in our prediction time frame. Using our example above our final target variable will be a value that is 0 if the value on 2008-06-09 is greater than 2008-06-25 or 1 if the situation is reversed.

##### Prediction horizon example

Below I combined the original dataset side by side with the shifted values & the prediction value to illustrate my description above.

In [ ]:
#prepare the data by shifting according to our desired horizon
pred_df = dji_df.shift(-10)['Close'].iloc[:-10]

example_df = pd.concat([dji_df['Close'].head(14), pred_df.head(14)],axis=1)
example_df.columns = ['Close','Close-shift']

example_df['Prediction'] = (example_df['Close-shift'] >= example_df['Close']).astype(int)

example_df

Just to better illustrate our variable set we've taken the closing price from 2008-06-09 compared it to 2008-06-25 (the 10th value in our DJIA data) and returned a 0 because the 2008-06-25 data was less than the closing price for 2006-06-09. Essentially the price has dropped from day 0 as compared to day 10.

##### Calculate the prediction variable 

Below we calculate the prediction variable based off the logic above. 

In [ ]:
values = [i for i in range(5,20) if i%5==0] #prediction horizons 5,10,15
names = ['pred_df_'+ str(v) for v in values] #names of the values based off of the horizons

pred_d = {name: 
             (dji_df.shift(-value)['Close'] >= dji_df['Close']).iloc[:-value].astype(int) 
             for name,value in zip(names,values)
        }

In [ ]:
#name the datasets
pred_df_5 = pred_d['pred_df_5']
pred_df_10 = pred_d['pred_df_10']
pred_df_15 = pred_d['pred_df_15']

print('5 day')
print(pred_df_5.value_counts())
print(FAT_BAR)
print('10 day')
print(pred_df_10.value_counts())
print(FAT_BAR)
print('15 day')
print(pred_df_15.value_counts())

##### Merge the DJIA data with the news headlines

In [ ]:
news_df.index = pd.to_datetime(news_df.index) #set news_df index to datetime

In [ ]:
names = ['combined_df_'+ str(v) for v in [i for i in range(5,20) if i%5==0]] 

combined = {name:
        news_df.merge(pred_d[[k for k in pred_d.keys()][i]], left_on='Date' , right_on='Date', how='right')
        for name, i in zip(names,range(len(names)))
    }

combined_df_5  = combined['combined_df_5']
combined_df_10 = combined['combined_df_10']
combined_df_15 = combined['combined_df_15']

combined_df_10.head()

##### Bull/Bear datasets

We may have more luck training the data on periods in time when the market is moving in a general trend. A bear market is characteristic of a downtrend and a bull is characterized as an upward trend. According to some economists the 2008 receission started in fall 2008 and ended in June 2009. Below I dilenate to dataframes "bull" and "bear" based on the dates above for use below. The idea being that the general market momentum in that discrete timeframe may produce higher prediction results.

In [ ]:
bull_names = ['bull_df_'+ str(v) for v in [i for i in range(5,20) if i%5==0]] 
bear_names = ['bear_df_'+ str(v) for v in [i for i in range(5,20) if i%5==0]] 

bull = {name:
        combined[[k for k in combined.keys()][i]][combined[[k for k in combined.keys()][i]].index >= '2009-07-01']
        for name, i in zip(bull_names,range(len(bull_names)))
    }

bear = {name:
        combined[[k for k in combined.keys()][i]][combined[[k for k in combined.keys()][i]].index <= '2009-05-01']
        for name, i in zip(bear_names,range(len(bear_names)))
    }

bull_df_5 = bull['bull_df_5']
bull_df_10 = bull['bull_df_10']
bull_df_15 = bull['bull_df_15']

bear_df_5 = bear['bear_df_5']
bear_df_10 = bear['bear_df_10']
bear_df_15 = bear['bear_df_15']

bear_df_5.head()

#### NLP feature engineering

##### Extracting features from text

To extract features for training the headlines we will use the bag of words model to train the headlines. Bag of words assigns a value to a word and counts the most frequent words in the dataset. Scikit learn provides a method of creating bag of words using countvectorizor, in which the columns are words and the rows are documents (headlines). 

Before we start though, we must clean up the text. This will consist of removing stopwords, making all the characters lowercase, stemming (finding the root: ask and asked), and converting to bigrams to help the algorithm predict what word is next.

In [ ]:
import gensim
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk import bigrams

STEMMER = SnowballStemmer("english", ignore_stopwords=True)

def preprocess(text: pd.Series, *args):
    text = text.apply(gensim.utils.simple_preprocess, min_len=3)
    sw = set(stopwords.words('english'))

    text = text.apply(lambda s: [w for w in s if w not in sw])
    text = text.apply(lambda s: [STEMMER.stem(w) for w in s])
    text = text.apply(lambda s: ['_'.join(x) for x in nltk.bigrams(s)] + s)

    return text

##### Preprocessing data...

Below is a quick snippet of what our data will look like once it has been pre processed. As you can see the data has been evaluated with the steps outlined above. I've printed out the 

In [ ]:
#bull_df_ = preprocess(bull_df.News)

In [ ]:
#bull_df_.iloc[0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 5000)

For the purposes of this notebook I chose 5000 features, and this was determined somewhat arbitrarily. I will determine empirically if I should grow/shrink the dataset after running my results.

Below is an example of the cleaned news dataset...

In [ ]:
#bull_['bull_df_5_']['News'].apply(lambda x: ', '.join(map(str, x)))

In [ ]:
#apply it to bull/bear datasets

bull_ = {name + '_':
     vectorizer.fit_transform(preprocess(bull[name].News).apply(lambda x: ', '.join(map(str, x))))
     for name in bull.keys()
    }

bear_ = {name + '_':
     vectorizer.fit_transform(preprocess(bear[name].News).apply(lambda x: ', '.join(map(str, x))))
     for name in bear.keys()
    }


bull_['bull_df_5_']

##### Naive Bayes model

First we will train on a Naive Bayes model

For the bull data we check what our odds are for choosing a 0 or 1 at random our as that is our baseline comparison tool against our model. If our model were to perform as good or worse than probability then we would know that we are not seeing anything interesting. As you can see below for the bull data we have to do better than ~40% for downs and ~60-63% for ups.

In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import roc_curve, auc

#roc plot

def roc_plot(fpr, tpr, roc_auc):
    # method I: plt
    import matplotlib.pyplot as plt
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [ ]:
print('odds of choosing a value at random')
print('')
print('horizon  down  up')

for d in bull.keys():
    x = bull[d]['Close'].value_counts()[0] + bull[d]['Close'].value_counts()[1]
    zero,one = bull[d]['Close'].value_counts()[0]/x, bull[d]['Close'].value_counts()[1]/x
    print(d, round(zero,2), round(one,2))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

clf = MultinomialNB()

In [ ]:
for d,i in zip(bull_.keys(),bull.keys()):               
    X,y = bull_[d], bull[i]['Close']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 465)
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    horizon = re.findall("[0-9]*$",i[-2:])[0]
    print(f'{d[:4]} {horizon} day prediction horizon')
    print(classification_report(y_test, y_pred))
    probs = clf.predict_proba(X_test)
    preds = probs[:,1]
    fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
    roc_auc = metrics.auc(fpr, tpr)
    roc_plot(fpr,tpr,roc_auc)

In [ ]:
print('odds of choosing a value at random')
print('')
print('horizon  down  up')

for d in bear.keys():
    x = bear[d]['Close'].value_counts()[0] + bear[d]['Close'].value_counts()[1]
    zero,one = bear[d]['Close'].value_counts()[0]/x, bear[d]['Close'].value_counts()[1]/x
    print(d, round(zero,2), round(one,2))

In [ ]:
for d,i in zip(bear_.keys(),bear.keys()):               
    X,y = bear_[d], bear[i]['Close']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 465)
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    horizon = re.findall("[0-9]*$",i[-2:])[0]
    print(f'{d[:4]} {horizon} day prediction horizon')
    print(classification_report(y_test, y_pred))
    probs = clf.predict_proba(X_test)
    preds = probs[:,1]
    fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
    roc_auc = metrics.auc(fpr, tpr)
    roc_plot(fpr,tpr,roc_auc)

In [ ]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier(n_estimators=100)

# We'll make 500 iterations, use 2-deep trees, and set our loss function.
params = {'n_estimators': 500,
          'max_depth': 2,
          'loss': 'deviance'}

# Initialize and fit the model.
ens = ensemble.GradientBoostingClassifier(**params)

In [ ]:
for d,i in zip(bull_.keys(),bull.keys()):               
    X,y = bull_[d], bull[i]['Close']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 465)
    ens.fit(X_train,y_train)
    y_pred = ens.predict(X_test)
    horizon = re.findall("[0-9]*$",i[-2:])[0]
    print(f'{d[:4]} {horizon} day prediction horizon')
    print(classification_report(y_test, y_pred))
    probs = ens.predict_proba(X_test)
    preds = probs[:,1]
    fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
    roc_auc = metrics.auc(fpr, tpr)
    roc_plot(fpr,tpr,roc_auc)

In [ ]:
for d,i in zip(bear_.keys(),bear.keys()):               
    X,y = bear_[d], bear[i]['Close']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 465)
    ens.fit(X_train,y_train)
    y_pred = ens.predict(X_test)
    horizon = re.findall("[0-9]*$",i[-2:])[0]
    print(f'{d[:4]} {horizon} day prediction horizon')
    print(classification_report(y_test, y_pred))
    probs = ens.predict_proba(X_test)
    preds = probs[:,1]
    fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
    roc_auc = metrics.auc(fpr, tpr)
    roc_plot(fpr,tpr,roc_auc)

##### Small sample from 4Q 2010

Here we take a small sample from the financial fourth quarter of 2010. As you can see from the graph below the market gained almost 1000 points during this time period. The motivation behind this is to see if we can predict better on a small period in time.

In [ ]:
plt.figure(figsize=(20, 6))

q = dji_df[(dji_df.index >= '2010-10-01') & (dji_df.index <= '2010-12-31')]['Close']

sns.tsplot(data=q)
plt.show()

In [ ]:
#establish the time period
market_4q = dji_df[(dji_df.index >= '2010-10-01') & (dji_df.index <= '2010-12-31')] 

#names of the values based off of the horizons
names = ['pred_df_'+ str(v) for v in [i for i in range(4,21)]] 

#create the shifted dataframes
pred_4q = {name: 
             (market_4q.shift(-value)['Close'] >= market_4q['Close']).iloc[:-value].astype(int) 
             for name,value in zip(names,range(len(names)))
        }

#merge headlines with with shifted dfs
combined = {name:
            news_df.merge(pred_4q[[k for k in pred_4q.keys()][i]], left_on='Date' , right_on='Date', how='right')
            for name, i in zip(names,range(len(names)))
        }

#NLP feature engineering
names_ = ['pred_df_'+ str(v) for v in [i for i in range(5,21)]]

combined_ = {name + '_':
             vectorizer.fit_transform(preprocess(combined[name].News).apply(lambda x: ', '.join(map(str, x))))
             for name in names_
        }

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from sklearn.utils import resample

nvb = MultinomialNB()

In [ ]:
for d,i in zip(combined_.keys(),[d for d in combined.keys()][1:]):               
    X,y = combined_[d], combined[i]['Close']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 465)
    
    y_t = pd.DataFrame(csr_matrix(y_train.values).toarray().reshape(-1,1))
    y_t['Close'] = pd.DataFrame(csr_matrix(y_train.values).toarray().reshape(-1,1)) #transpose
    X = pd.concat([pd.DataFrame(X_train.toarray()), y_t['Close']], axis=1) #convert from sparse matrix
    
    #upsample
    down,up = X[X.Close==0],X[X.Close==1] 
    down_upsampled = resample(down,
                              replace=True, # sample with replacement
                              n_samples=len(up), # match number in majority class
                              random_state=27) # reproducible results

    # combine majority and upsampled minority
    upsampled = pd.concat([down_upsampled, up])
    
    nvb.fit(upsampled.loc[:,~upsampled.columns.isin(['Close'])], upsampled['Close'])
    y_pred = nvb.predict(X_test)
    horizon = re.findall("[0-9]*$",i[-2:])[0]
    print(f'{d[:4]} {horizon} day prediction horizon')
    print(classification_report(y_test, y_pred))
    probs = nvb.predict_proba(X_test)
    preds = probs[:,1]
    fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
    roc_auc = metrics.auc(fpr, tpr)
    roc_plot(fpr,tpr,roc_auc)

In [ ]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier(n_estimators=100)

# We'll make 500 iterations, use 2-deep trees, and set our loss function.
params = {'n_estimators': 500,
          'max_depth': 2,
          'loss': 'deviance'}

# Initialize and fit the model.
ens = ensemble.GradientBoostingClassifier(**params)

In [ ]:
for d,i in zip(combined_.keys(),[d for d in combined.keys()][1:]):               
    X,y = combined_[d], combined[i]['Close']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 465)
    
    y_t = pd.DataFrame(csr_matrix(y_train.values).toarray().reshape(-1,1))
    y_t['Close'] = pd.DataFrame(csr_matrix(y_train.values).toarray().reshape(-1,1)) #transpose
    X = pd.concat([pd.DataFrame(X_train.toarray()), y_t['Close']], axis=1) #convert from sparse matrix
    
    #upsample
    down,up = X[X.Close==0],X[X.Close==1] 
    down_upsampled = resample(down,
                              replace=True, # sample with replacement
                              n_samples=len(up), # match number in majority class
                              random_state=27) # reproducible results

    # combine majority and upsampled minority
    upsampled = pd.concat([down_upsampled, up])
    
    rfc.fit(upsampled.loc[:,~upsampled.columns.isin(['Close'])], upsampled['Close'])
    y_pred = rfc.predict(X_test)
    horizon = re.findall("[0-9]*$",i[-2:])[0]
    print(f'{d[:4]} {horizon} day prediction horizon')
    print(classification_report(y_test, y_pred))
    probs = rfc.predict_proba(X_test)
    preds = probs[:,1]
    fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
    roc_auc = metrics.auc(fpr, tpr)
    roc_plot(fpr,tpr,roc_auc)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

svc = SVC(C=1.0, kernel='linear', class_weight='balanced', probability=True)

In [ ]:
for d,i in zip(combined_.keys(),[d for d in combined.keys()][1:]):               
    X,y = combined_[d], combined[i]['Close']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 465)
    svc.fit(X_train,y_train)
    y_pred = svc.predict(X_test)
    horizon = re.findall("[0-9]*$",i[-2:])[0]
    print(f'{d[:4]} {horizon} day prediction horizon')
    print(classification_report(y_test, y_pred))
    probs = svc.predict_proba(X_test)
    preds = probs[:,1]
    fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
    roc_auc = metrics.auc(fpr, tpr)
    roc_plot(fpr,tpr,roc_auc)

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

clf1 = SVC(C=1.0, kernel='linear', class_weight='balanced', probability=True)
clf2 = GaussianNB()
clf3 = RandomForestClassifier(n_estimators=50, random_state=1)

eclf = VotingClassifier(estimators=[('svc', clf1), 
                                    ('gnb', clf2), 
                                    ('rfc', clf3)], 
                                    voting='soft')

In [ ]:
for d,i in zip(combined_.keys(),[d for d in combined.keys()][1:]):               
    X,y = combined_[d].toarray(), combined[i]['Close']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 465)
    eclf.fit(X_train,y_train)
    y_pred = eclf.predict(X_test)
    horizon = re.findall("[0-9]*$",i[-2:])[0]
    print(f'{d[:4]} {horizon} day prediction horizon')
    print(classification_report(y_test, y_pred))
    probs = eclf.predict_proba(X_test)
    preds = probs[:,1]
    fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
    roc_auc = metrics.auc(fpr, tpr)
    roc_plot(fpr,tpr,roc_auc)

##### 3Q 2008

In [ ]:
plt.figure(figsize=(20, 6))

q = dji_df[(dji_df.index >= '2008-07-01') & (dji_df.index <= '2008-10-30')]['Close']

sns.tsplot(data=q)
plt.show()

In [ ]:
#establish the time period
market_3q = dji_df[(dji_df.index >= '2008-07-01') & (dji_df.index <= '2008-10-30')] 

#names of the values based off of the horizons
names = ['pred_df_'+ str(v) for v in [i for i in range(4,21)]] 

#create the shifted dataframes
pred_3q = {name: 
             (market_3q.shift(-value)['Close'] >= market_3q['Close']).iloc[:-value].astype(int) 
             for name,value in zip(names,range(len(names)))
        }

#merge headlines with with shifted dfs
combined = {name:
            news_df.merge(pred_3q[[k for k in pred_3q.keys()][i]], left_on='Date' , right_on='Date', how='right')
            for name, i in zip(names,range(len(names)))
        }

#NLP feature engineering
names_ = ['pred_df_'+ str(v) for v in [i for i in range(5,21)]]

combined_ = {name + '_':
             vectorizer.fit_transform(preprocess(combined[name].News).apply(lambda x: ', '.join(map(str, x))))
             for name in names_
        }

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from sklearn.utils import resample

nvb = MultinomialNB()

In [ ]:
for d,i in zip(combined_.keys(),[d for d in combined.keys()][1:]):               
    X,y = combined_[d], combined[i]['Close']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 465)
    
    y_t = pd.DataFrame(csr_matrix(y_train.values).toarray().reshape(-1,1))
    y_t['Close'] = pd.DataFrame(csr_matrix(y_train.values).toarray().reshape(-1,1)) #transpose
    X = pd.concat([pd.DataFrame(X_train.toarray()), y_t['Close']], axis=1) #convert from sparse matrix
    
    #upsample
    down,up = X[X.Close==0],X[X.Close==1] 
    down_upsampled = resample(down,
                              replace=True, # sample with replacement
                              n_samples=len(up), # match number in majority class
                              random_state=27) # reproducible results

    # combine majority and upsampled minority
    upsampled = pd.concat([down_upsampled, up])
    
    nvb.fit(upsampled.loc[:,~upsampled.columns.isin(['Close'])], upsampled['Close'])
    y_pred = nvb.predict(X_test)
    horizon = re.findall("[0-9]*$",i[-2:])[0]
    print(f'{d[:4]} {horizon} day prediction horizon')
    print(classification_report(y_test, y_pred))
    probs = nvb.predict_proba(X_test)
    preds = probs[:,1]
    fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
    roc_auc = metrics.auc(fpr, tpr)
    roc_plot(fpr,tpr,roc_auc)

In [ ]:
for d,i in zip(combined_.keys(),[d for d in combined.keys()][1:]):               
    X,y = combined_[d], combined[i]['Close']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 465)
    
    y_t = pd.DataFrame(csr_matrix(y_train.values).toarray().reshape(-1,1))
    y_t['Close'] = pd.DataFrame(csr_matrix(y_train.values).toarray().reshape(-1,1)) #transpose
    X = pd.concat([pd.DataFrame(X_train.toarray()), y_t['Close']], axis=1) #convert from sparse matrix
    
    #upsample
    down,up = X[X.Close==0],X[X.Close==1] 
    down_upsampled = resample(down,
                              replace=True, # sample with replacement
                              n_samples=len(up), # match number in majority class
                              random_state=27) # reproducible results

    # combine majority and upsampled minority
    upsampled = pd.concat([down_upsampled, up])
    
    rfc.fit(upsampled.loc[:,~upsampled.columns.isin(['Close'])], upsampled['Close'])
    y_pred = rfc.predict(X_test)
    horizon = re.findall("[0-9]*$",i[-2:])[0]
    print(f'{d[:4]} {horizon} day prediction horizon')
    print(classification_report(y_test, y_pred))
    probs = rfc.predict_proba(X_test)
    preds = probs[:,1]
    fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
    roc_auc = metrics.auc(fpr, tpr)
    roc_plot(fpr,tpr,roc_auc)

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

clf1 = SVC(C=1.0, kernel='linear', class_weight='balanced', probability=True)
clf2 = GaussianNB()
clf3 = RandomForestClassifier(n_estimators=50, random_state=1)

eclf = VotingClassifier(estimators=[('svc', clf1), 
                                    ('gnb', clf2), 
                                    ('rfc', clf3)], 
                                    voting='soft')

In [ ]:
for d,i in zip(combined_.keys(),[d for d in combined.keys()][1:]):               
    X,y = combined_[d].toarray(), combined[i]['Close']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 465)
    eclf.fit(X_train,y_train)
    y_pred = eclf.predict(X_test)
    horizon = re.findall("[0-9]*$",i[-2:])[0]
    print(f'{d[:4]} {horizon} day prediction horizon')
    print(classification_report(y_test, y_pred))
    probs = eclf.predict_proba(X_test)
    preds = probs[:,1]
    fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
    roc_auc = metrics.auc(fpr, tpr)
    roc_plot(fpr,tpr,roc_auc)

##### Combine 4Q & 3Q

In [ ]:
#establish the time period
market_3q = dji_df[(dji_df.index >= '2008-07-01') & (dji_df.index <= '2008-10-30')] 
market_4q = dji_df[(dji_df.index >= '2010-10-01') & (dji_df.index <= '2010-12-31')] 

df = pd.concat([market_3q,market_4q],axis=0)

#names of the values based off of the horizons
names = ['pred_df_'+ str(v) for v in [i for i in range(4,21)]] 

#create the shifted dataframes
pred_df = {name: 
             (df.shift(-value)['Close'] >= df['Close']).iloc[:-value].astype(int) 
             for name,value in zip(names,range(len(names)))
        }

#merge headlines with with shifted dfs
combined = {name:
            news_df.merge(pred_df[[k for k in pred_df.keys()][i]], left_on='Date' , right_on='Date', how='right')
            for name, i in zip(names,range(len(names)))
        }

#NLP feature engineering
names_ = ['pred_df_'+ str(v) for v in [i for i in range(5,21)]]

combined_ = {name + '_':
             vectorizer.fit_transform(preprocess(combined[name].News).apply(lambda x: ', '.join(map(str, x))))
             for name in names_
        }

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

clf1 = SVC(C=1.0, kernel='linear', class_weight='balanced', probability=True)
clf2 = GaussianNB()
clf3 = RandomForestClassifier(n_estimators=50, random_state=1)

eclf = VotingClassifier(estimators=[('svc', clf1), 
                                    ('gnb', clf2), 
                                    ('rfc', clf3)], 
                                    voting='soft')

In [ ]:
for d,i in zip(combined_.keys(),[d for d in combined.keys()][1:]):               
    X,y = combined_[d].toarray(), combined[i]['Close']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 465)
    eclf.fit(X_train,y_train)
    y_pred = eclf.predict(X_test)
    horizon = re.findall("[0-9]*$",i[-2:])[0]
    print(f'{d[:4]} {horizon} day prediction horizon')
    print(classification_report(y_test, y_pred))
    probs = eclf.predict_proba(X_test)
    preds = probs[:,1]
    fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
    roc_auc = metrics.auc(fpr, tpr)
    roc_plot(fpr,tpr,roc_auc)

In [ ]:
print('odds of choosing a value at random')
print('')
print('horizon  down  up')

for d in combined.keys():
    x = bear[d]['Close'].value_counts()[0] + bear[d]['Close'].value_counts()[1]
    zero,one = bear[d]['Close'].value_counts()[0]/x, bear[d]['Close'].value_counts()[1]/x
    print(d, round(zero,2), round(one,2))

##### Conclusion and warning about extreme events

### Precision v. recall tradeoff 

We appeared to have our best success on a small timeframe (4Q 2010) with prediction of upward movements on a 15 day horizon with an ensamble model depth of 2 and 500 iterations. Possible explanations could be that positive momentum leads to overall market optimism and further investment in the market (it's never going to go down!). Why a 15-day horizon produces slightly better results will warrent further investigation.

Fundamentally we know that during a bull market we have more buyers than sellers thus the price of an asset will go up. In most cases (the rational case) you will buy an asset if you expect the price of that asset to gain value over time. As many famous investors have observed that the market can be a general evaluation of investor sentiment, we can speculate that optimism dominates during bull markets and the opposite during bear markets. The goal of this exercise was to see if we could capture that sentiment expressed through headlines.

Ultimately the movement of a stock is a stochastic process and the economic change to a recession (where is the top v. bottom of a price) is something that this model lacks. We cannot say with certainty when the market will take a nosedive. Thus a more sophisticated model would consider not only predicted price movements but also magnitude of downside risk. Although our model predicted decently for 2010 Q4, the potential to wipe out small incremental gains over the course of many days in one day exists (See the Turkey problem/problem of induction). Additionally, this illustrates why we can't normalize the stock data. Predictions of 95% confidence of market moves may appear to be very favorable but tail events can have effects on your financial position.

##### Future work

Further analysis can be done on features have a greater effect during bull and bear markets. We can accomplish this through other methods of extracting word features such as TF-IDF and word2vec for example. Additionally we can applying other methods of classification the data such as neural networks. Finally we can generate what quant's call technical indicators to assist in the training of the models. There are numerous technical indicators that seek to measure price momentum.

Another point that must be investigated is how can we predict market movements that go agains the macro-economic environment, such as down movements during a bull market and vice-versa. Our model did well predicting the market was going up during a bull market, but how much was this due to chance.

Given more time, further investigations can be done into the optimal prediction horizon using a loop to create a dataframe for a given period of time.